# <center> <img src="./img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Proyecto Final: Website Activity** </center>
---

<center>

**Equipo McQueen**: Marco Albanese, Vicente Siloe

**Carrera**: Ingeniería en Sistemas Computacionales

**Fecha**: 13 de mayo del 2025

**Profesor**: Pablo Camarillo Ramirez

</center>

---
### <center> **Primavera 2025** </center>
---

### **Introducción y definición del problema**

El procesamiento de datos en aplicaciones es esencial hoy en día para una multitud de usos y propósitos que proveen beneficios tanto a usuarios como empresas. En este proyecto, creado al culminar el curso de "Procesamiento de datos masivos", se pretende crear una aplicación que integre los conocimientos adquiridos en dicho curso. A través de una *pipeline* de *Big Data*, se busca producir, consumir, analizar y procesar datos de la actividad (tráfico) observada en una aplicación web.

Los datos involucrados representan:
- Interacciones de usuario
- Páginas visitadas
- Clicks
- Dispositivos utilizados
- Etc.

Asimismo, se entrena un modelo de aprendizaje automático (*machine learning*) en base a los datos recolectados para predecir comportamientos de usuario, utilizando regresión logística. Los resultados de este modelo se presentan al usuario final a través de un *dashboard* de visualización de datos.

Para la realización de este proyecto, se utilizaron tecnologías como **Jupyter Notebooks**, **Python**, **PySpark**, **Apache Kafka** y **Power BI**. En secciones posteriores, se describe con mayor detalle la arquitectura, implementación y funcionamiento del sistema.

### **Arquitectura del sistema**

La arquitectura del sistema es la siguiente:

1. **Productores Kafka:** encargados de generar datos de actividad/tráfico web. Se utilizan dos acorde a la cantidad de integrantes en el equipo y dichos datos son publicados a dos tópicos individuales.
2. **Kafka Broker:** intermediario para conectar productores con consumidor, donde se crean los tópicos necesarios.
3. **Consumidor PySpark:** consumidor de los datos en tiempo real, guardando activamente en formato Parquet para formar un data lake.
4. **Modelo de aprendizaje máquina (Machine Learning):** modelo de K-means para la segmentación de usuarios en base a su comportamiento navegando la web.
5. **Visualización de datos:** se incluye un dashboard de visualización en base a los resultados obtenidos por el modelo de *machine learning*, realizado en Power BI.

A continuación se presenta un diagrama visual de dicha arquitectura:


# <img src="./img/system_architecture.png" alt="System Architecture">

### **Justificación de las 5Vs**

- **Volumen:** En una ejecución de 60 segundos, se generan aproximadamente 50 archivos en formato Parquet. Seleccionando dichos archivos y verificando sus propiedades, obtenemos un tamaño de 370 KB. En promedio, se observa un tamaño de 7.4 KB por archivo. Teniendo esta información, podemos generar la tabla de crecimiento:

<center>

| **Periodo de tiempo** | **Datos procesados** |
|-----------------------|----------------------|
| 1 segundo             | 7.4 KB               |
| 1 minuto              | 444 KB               |
| 1 hora                | 26.64 MB             |
| 1 día                 | 639.36 MB            |
| 1 año                 | 233.1 GB             |

</center>

- **Velocidad:** Este valor no pudo ser incluido en esta justificación. La explicación del problema se presenta en la sección de "Problemas encontrados"

- **Variedad:** El esquema de datos producidos por el tráfico de la página web es el siguiente:

<center>


| **Campo**           | **Tipo de dato** |
|---------------------|------------------|
| event_id            | String           |
| user_id             | String           |
| page_url            | String           |
| timestamp           | Timestamp        |
| action_type         | String           |
| browser             | String           |
| timezone            | String           |
| device_type         | String           |
| click_coordinates_x | Integer          |
| click_coordinates_y | Integer          |
| session_id          | String           |
| ip_address          | String           |

</center>


- **Veracidad:** Los datos generados para cada campo se asemejan a la realidad gracias al uso de la librería Faker. Por ejemplo, los IDs son generados en formato UUID de 128 bits, *timezone* representa una zona horaria real, *ip_address* sigue el formato de IPv4, etc. 

- **Valor:** los datos almacenados brindan información valiosa acerca de usuarios, sus intereses en páginas web, navegadores más comunes, tipos de dispositivos (ej. si se debería poner más esfuerzo en una página amigable a dispositivos móviles), la región a la que pertenecen, etc.

### **Detalles de implementación**

- Para el levantamiento del broker de Kafka como también PySpark, se utilizaron contenedores creados a través de Docker Compose, una herramienta para correr aplicaciones multi-contenedor.

- Para la creación de los productores de datos, se utilizaron dos Jupyter Notebooks localizados en la carpeta `lib`, llamados `producer_a.ipynb` y `producer_b.ipynb`. Estos mantienen parecido al ejemplo localizado en `notebooks/examples/producers/kafka_producer.ipynb`. 
  - A su vez, ambos productores importan una función auxiliar de un script de Python llamado `website_activity.py`, localizado en la carpeta `lib`. Este script utiliza la librería Faker para generar datos aleatorios.

La implementación de esta aplicación sigue una estructura similar a laboratorios vistos en clase. Notablemente, se toma como inspiración a los siguientes archivos:

- `example_structured_streaming_files_traffic_data.ipynb`
- `example_structured_streaming_files.ipynb`
- `example_structured_streaming_kafka_watermarking.ipynb`
- `example_structured_streaming_kafka.ipynb`

Para el modelo de Machine Learning, se utilizó K-means para crear una segmentación de usuarios que pueda ser utilizada para identificar diferentes comportamientos (ej. usuarios que más hacen click, usuarios que más scrollean, etc). La elección del valor de `k` fue realizada de manera arbitraria. Se debe mencionar que este no es un modo correcto o justificable para elegir este valor, por lo que se requeriría un nuevo análisis para identificar un mejor o más adecuado valor. Métodos disponibles para este fin pueden ser consultados en https://en.wikipedia.org/wiki/K-means_clustering

Se tomó como referencia el siguiente ejemplo:

- `example_k_means.ipynb`

### **Resultados y evaluación**

Al evaluar la puntuación de la silueta (silhouette) del modelo, se obtuvo un valor de 0.59, el cual está más cercano a 1 que a 0, por lo que se alcanzó una separación regular de los clusters generados. Encontrando un mejor valor de `k` para utilizar, se podría obtener una mejor puntuación para la silueta. Al exporar los resultados de las predicciones a CSV para su uso en Power BI, se creó un dashboard sencillo. Este muestra información acerca de los usuarios como también estadísticas relevantes del tráfico web observado.

Por un lado, se pueden observar la cantidad de clicks que se reciben por tipo de dispositivo, asi como la cantidad de *scrolleos* por cada tipo. Por otro lado, se observa la cantidad de usuarios que cae (en promedio) sobre un clúster en particular. Estos clusters representan un "tipo de usuario", donde se tiene, por ejemplo, a usuarios que hacen muchos clicks (cluster 3), como también usuarios que visitan muchas páginas (cluster 2).


# <center> <img src="./img/sample_dashboard.png" width="50%"> </center>

### **Problemas encontrados**

La justificación de "Velocidad" se intentó generar a través de la implementación de un QueryListener, similar al de la actividad realizada para datos de tráfico. La implementación propuesta era la siguiente:

```python
class MyQueryListener(StreamingQueryListener):
    def onQueryStarted(self, event):
        print(f"Query started: {event.id}")

    def onQueryProgress(self, event):
        print(f"[{event.timestamp}]")
        print(f"Input Rows Per Second: {event.inputRowsPerSecond}")
        print(f"Processed Rows Per Second: {event.processedRowsPerSecond}")
        print(f"Num Input Rows: {event.numInputRows}")

    def onQueryTerminated(self, event):
        print(f"Query terminated: {event.id}")

```

Al tratar de utilizar este código, encontramos que saltaban errores debido al ID del evento, así como las propiedades que se intentan consultar de dicho evento (`inputRowsPerSecond`, etc). Se intentó diagnosticar más a fondo pero no se llegó a una solución. Por ello, el valor para justificar este campo no se incluye.

### **Conclusión**

La realización de este proyecto representa la culminación de los conocimientos adquiridos a lo largo del curso de Procesamiento de datos masivos. Se lograron integrar varios componentes de distintas lecturas y laboratorios de clase, principalmente a través de PySpark y Kafka. Por un lado, la parte más interesante ha sido la producción y consumo de datos para su posterior procesamiento. Por otro lado, identificamos que, como trabajo futuro, se podría mejorar el valor dado por el segmento de *machine learning*. Consideramos que esta parte, aunque relativamente fácil de implementar a nivel código, requiere de mayor planeamiento y decisiones acertadas. 

---

### **Código: Spark + Kafka**
---

Previo a la ejecución del siguiente código, los contenedores de Docker para el cluster de Spark y el cluster de Kafka deben estar activos.

Para ello, se puede utilizar los comandos:

`docker compose up --scale spark-worker=2 -d`

`docker compose up -d`

En los directorios `spark_cluster` y `kafka_cluster` respectivamente. Nótese que se requerirán distintos IDs de contenedores en los bloques de código pertinentes.

#### Encontrar instalación de Spark

In [3]:
import findspark
findspark.init()

#### Creación de la conexión con el cluster de Spark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Final-Project-Equipo-McQueen") \
    .master("spark://a6acd6b0849a:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()

sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8c65f715-28d5-46da-84ca-a63765f06ea1;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

#### Creación de tópicos de Kafka

Los siguientes comandos deben ser ejecutados en la terminal para que los productores puedan enviar mensajes a los tópicos:

```
docker exec -it d6fae88657b9 /opt/kafka/bin/kafka-topics.sh --create --zookeeper zookeeper:2181 --replication-factor 1 --partitions 1 --topic producer-a

docker exec -it d6fae88657b9 /opt/kafka/bin/kafka-topics.sh --create --zookeeper zookeeper:2181 --replication-factor 1 --partitions 1 --topic producer-b
```

#### Ejecución de productores

Para inicializar a los productores, ejecute las respectivas celdas en los Jupyter Notebooks (`producer-a.ipynb`, `producer-b.ipynb`) bajo el directorio `spark_cluster/notebooks/lib/equipo_mcqueen/`

#### Creación del Kafka stream

In [9]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "d6fae88657b9:9093") \
                .option("subscribe", "producer-a,producer-b") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



#### Transformar datos en cadenas (strings)

In [10]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

#### Creación del query (sink)

In [11]:
query = kafka_df \
                .writeStream \
                .trigger(processingTime="3 seconds") \
                .outputMode("append") \
                .format("parquet") \
                .option("path", "/home/jovyan/notebooks/data/") \
                .option("checkpointLocation", "/home/jovyan/checkpoint") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(60)

25/05/16 14:10:02 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/16 14:10:02 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/16 14:10:05 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3181 milliseconds


False

#### Terminación de query y context

In [12]:
query.stop()

In [14]:
sc.stop()

### **Código: Machine Learning**
---

#### Encontrar instalación de Spark

In [15]:
import findspark
findspark.init()

#### Creación de la conexión con el cluster de Spark

In [16]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Equipo-McQueen-KMeans") \
    .master("spark://a6acd6b0849a:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

#### Preparación de datos

In [17]:
website_activity_df = spark.read.parquet("/home/jovyan/notebooks/data/")
website_activity_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- value_str: string (nullable = true)



#### Verificación de datos generados

In [18]:
website_activity_df.select("value_str").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value_str                                                                                                                                                                                                                                                                                                                                                                                                

#### Creación de esquema

In [19]:
from equipo_mcqueen.spark_utils import SparkUtils

website_activity_data = [
    ("user_id", "StringType"),
    ("action_type", "StringType"),
    ("device_type", "StringType"),
    ("click_coordinates_x", "IntegerType"),
    ("click_coordinates_y", "IntegerType")
]

activity_schema = SparkUtils.generate_schema(website_activity_data)

#### Extracción de datos para modelo

In [20]:
from pyspark.sql.functions import from_json, col

df = website_activity_df.withColumn("parsed_data", from_json(col("value_str"), activity_schema))
activity_df = df.select("parsed_data.*")
activity_df.printSchema()
activity_df.show(truncate=False)

root
 |-- user_id: string (nullable = true)
 |-- action_type: string (nullable = true)
 |-- device_type: string (nullable = true)
 |-- click_coordinates_x: integer (nullable = true)
 |-- click_coordinates_y: integer (nullable = true)

+------------------------------------+-----------+-----------+-------------------+-------------------+
|user_id                             |action_type|device_type|click_coordinates_x|click_coordinates_y|
+------------------------------------+-----------+-----------+-------------------+-------------------+
|8ca3c9d4-09e2-4870-a435-b63ec051e984|view       |tablet     |1458               |970                |
|b5eb7f2d-e798-4879-9888-71ff1290bee7|scroll     |tablet     |1386               |967                |
|0486f014-9604-4737-a06d-d92c7a8f2679|scroll     |tablet     |1769               |882                |
|333e70e4-23e2-4e9f-95c0-136a7de2dfcd|click      |tablet     |1477               |1064               |
|15a03770-2724-47be-8ecb-f4f35fdf325c|click 

#### Agrupación y agregación de datos

In [21]:
from pyspark.sql.functions import first, count, avg, when

user_features = activity_df.groupBy("user_id").agg(first("device_type").alias("device_type"),
                                                   count(when(col("action_type") == "click", 1)).alias("click_count"),
                                                   count(when(col("action_type") == "scroll", 1)).alias("scroll_count"),
                                                   count(when(col("action_type") == "view", 1)).alias("view_count"),
                                                   avg("click_coordinates_x").alias("avg_click_coordinates_x"),
                                                   avg("click_coordinates_y").alias("avg_click_coordinates_y"))

user_features.show(truncate=False)

+------------------------------------+-----------+-----------+------------+----------+-----------------------+-----------------------+
|user_id                             |device_type|click_count|scroll_count|view_count|avg_click_coordinates_x|avg_click_coordinates_y|
+------------------------------------+-----------+-----------+------------+----------+-----------------------+-----------------------+
|00c5f9c4-1a02-4df9-833a-6cd1f503e85d|tablet     |1          |0           |0         |1481.0                 |663.0                  |
|01887bf6-3d6c-4cc9-a632-88ec548dc0b2|desktop    |1          |0           |0         |281.0                  |499.0                  |
|0486f014-9604-4737-a06d-d92c7a8f2679|tablet     |0          |1           |0         |1769.0                 |882.0                  |
|0556ddae-d320-4860-b27b-0af5de54ad39|mobile     |0          |1           |0         |1194.0                 |136.0                  |
|05855a5d-a9ee-4f52-9c9b-0c1e15240b58|tablet     |0    

#### Creación de vector

In [22]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

numerical_features = [
    "click_count",
    "scroll_count",
    "view_count", 
    "avg_click_coordinates_x",
    "avg_click_coordinates_y"
]

assembler = VectorAssembler(inputCols=numerical_features, outputCol="features")

assembled_data = assembler.transform(user_features)

#### Creación de Scaler

In [23]:
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withStd=True,
    withMean=True
)

scaler_model = scaler.fit(assembled_data)
scaled_data = scaler_model.transform(assembled_data)

#### Creación de modelo

In [24]:
from pyspark.ml.clustering import KMeans

k_value = 5

kmeans = KMeans(
    featuresCol="scaled_features",
    k=k_value,
    seed=42
)

#### Entrenamiento

In [25]:
model = kmeans.fit(scaled_data)

25/05/16 14:13:49 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/16 14:13:49 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


#### Predicciones

In [26]:
predictions = model.transform(scaled_data)

#### Evaluación

In [27]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator(
    featuresCol="scaled_features",
    predictionCol="prediction",
    metricName="silhouette"
)

silhouette = evaluator.evaluate(predictions)
print(f"Silhouette score = {silhouette}")



Silhouette score = 0.5435756098196128


In [28]:
predictions.select("user_id", "device_type", "prediction").show()

centers = model.clusterCenters()
print("Cluster Centers (Scaled Features):")
for i, center in enumerate(centers):
    print(f"Cluster {i}: {center}")

predictions.groupBy("prediction").count().show()


predictions.groupBy("prediction").agg(
    avg("click_count").alias("avg_clicks"),
    avg("scroll_count").alias("avg_scrolls"),
    avg("view_count").alias("avg_views"),
    avg("avg_click_coordinates_x").alias("avg_x"),
    avg("avg_click_coordinates_y").alias("avg_y"),
    count("device_type").alias("count")
).show()

+--------------------+-----------+----------+
|             user_id|device_type|prediction|
+--------------------+-----------+----------+
|00c5f9c4-1a02-4df...|     tablet|         3|
|01887bf6-3d6c-4cc...|    desktop|         2|
|0486f014-9604-473...|     tablet|         1|
|0556ddae-d320-486...|     mobile|         1|
|05855a5d-a9ee-4f5...|     tablet|         1|
|05a5bb54-cb3a-4d9...|    desktop|         1|
|06cd4114-ae93-437...|     mobile|         1|
|070f3006-6da5-413...|    desktop|         4|
|075d5f66-3e32-4c3...|     mobile|         1|
|07913434-6c0a-492...|    desktop|         1|
|08bbd8c4-8865-494...|    desktop|         1|
|08ce2077-c08d-448...|     mobile|         1|
|0c89af99-e70f-40d...|     tablet|         4|
|0cc3a0bb-1f9a-4aa...|     mobile|         1|
|0df68528-af6d-4e8...|     mobile|         4|
|10de57d9-e13a-476...|     tablet|         2|
|1413e768-1b86-42e...|     tablet|         1|
|15a03770-2724-47b...|     mobile|         3|
|163f5242-44da-496...|     tablet|

+----------+----------+-----------+---------+------------------+------------------+-----+
|prediction|avg_clicks|avg_scrolls|avg_views|             avg_x|             avg_y|count|
+----------+----------+-----------+---------+------------------+------------------+-----+
|         3|       1.0|        0.0|      0.0|1620.6470588235295| 689.1176470588235|   17|
|         0|       0.0|        0.0|      1.0|1389.8666666666666| 637.9333333333333|   30|
|         4|       0.0|        0.0|      1.0| 602.7142857142857|428.42857142857144|   21|
|         1|       0.0|        1.0|      0.0|1011.7450980392157| 630.4313725490196|   51|
|         2|       1.0|        0.0|      0.0| 643.1428571428571| 571.3809523809524|   21|
+----------+----------+-----------+---------+------------------+------------------+-----+



In [31]:
sc.stop()

### **Visualización de datos: Power BI**
---

#### Conversión a CSV (user clusters)

In [29]:
user_clusters = predictions.select(
    "user_id", 
    "device_type",
    "click_count", 
    "scroll_count", 
    "view_count", 
    "avg_click_coordinates_x", 
    "avg_click_coordinates_y",
    "prediction"
).withColumnRenamed("prediction", "cluster")

user_clusters.write.mode("overwrite").option("header", "true").csv("/home/jovyan/notebooks/data/user_clusters")

#### Conversión a CSV (cluster stats)

In [30]:
cluster_stats = predictions.groupBy("prediction").agg(
    count("user_id").alias("user_count"),
    avg("click_count").alias("avg_clicks"),
    avg("scroll_count").alias("avg_scrolls"),
    avg("view_count").alias("avg_views"),
    avg("avg_click_coordinates_x").alias("avg_x"),
    avg("avg_click_coordinates_y").alias("avg_y")
).withColumnRenamed("prediction", "cluster")

cluster_stats.write.mode("overwrite").option("header", "true").csv("/home/jovyan/notebooks/data/cluster_stats")

#### Dashboard de ejemplo

# <center> <img src="./img/sample_dashboard.png" width="50%"> </center>